<!--<h1 style="font-size:40px; font-family:Verdana" align="center"> UDS-Club Workshop </h1> -->
<h2 style="font-size:34px; font-family:Verdana" align="center"> Char-based Recurrent and Convotional Neural Networks for Text Classification </h2>
<img src='http://i.piccy.info/i9/666d78be04fbcf04fdb321d5953d1fa5/1492256847/123248/1137898/ua_parrots.jpg'/>
<h4 style="font-size:18px; font-family:Verdana" align="right"> by Vitaliy Radchenko <br> <pre>    2017-04-23</pre> </h4>

<hr style="height: 1px; background-color: #808080">
## Table of Contents

<ol>
    <li style="font-size:20px; font-family:Verdana">[Embedding approach](#1)</li>
    <li style="font-size:20px; font-family:Verdana">[OHE approach](#2)</li>
    <li style="font-size:20px; font-family:Verdana">[Different model architectures](#3)</li>
</ol>
<hr style="height: 1px; background-color: #808080">
## 1. Prerequisites

#### Installation-based (must be already on-board, see your Docker container)
<ol>
    <li>Python 3.x (or Anaconda3 for Python 3.5, https://www.continuum.io/downloads)</li>
    <li>Scikit-learn 0.18.x (pip install scikit-learn==0.18.1, http://scikit-learn.org/)</li>
    <li>TextBlob (pip install textblob, https://textblob.readthedocs.io/en/dev/)</li>
    <li>NLTK (pip install nltk, http://www.nltk.org/)
    <li>Pandas / NumPy / SciPy</li>
</ol>

#### Knowledge-based
<ol>
    <li>Basics of numpy</li>
    <li>Embeddings</li>
    <li>Recurrent Neural Networks</li>
    <li>Convolutional Neural Networks</li>
</ol>

From the first glance char-based models are the same as word-based models. 

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

BASE_DIR = '../'
TEXT_DATA_DIR = BASE_DIR + 'data/'
TEXT_DATA_FILE = "movie_reviews.csv"
HEADER = True

MAX_SEQUENCE_LENGTH = 1024
VALIDATION_SPLIT = 0.06552
RANDOM_SEED = 42

def load_data():
    x = []
    y = []
    with open(os.path.join(TEXT_DATA_DIR, TEXT_DATA_FILE), "r") as f:
        if HEADER:
            _ = next(f)
        for line in f:
            temp_y, temp_x = line.rstrip("\n").split(",", 1)
            x.append(temp_x)
            y.append(temp_y)

    return x, y

#loading data
data, labels = load_data()
labels = np.asarray(labels, dtype='int8')

#split data on train and validation sets
data_train, data_val, labels_train, labels_val = \
    train_test_split(data, np.asarray(labels, dtype='int8'),
                     test_size=VALIDATION_SPLIT, random_state=RANDOM_SEED, stratify=labels)

Next step will be to replace letters with its indexes. 

**Your task** is to create a vocabulary (include all lowercase english letters, digits, punctuation) and replace chars with indexes. Use library `string` and `keras.preprocessing.sequence.pad_sequences`

In [2]:
import string
from keras.preprocessing.sequence import pad_sequences

def create_vocab_set():
    
    #1. YOUR CODE
    return vocab, vocab_size

def text2sequence(text, vocab):
    temp = []
    #2. YOUR CODE
    return temp

vocab, vocab_size = create_vocab_set()

X_train = text2sequence(data_train, vocab)
X_val = text2sequence(data_val, vocab)

#3. YOUR CODE
X_train = 
X_val = 
y_train, y_test = to_categorical(labels_train), to_categorical(labels_test)

SyntaxError: invalid syntax (<ipython-input-2-ad5c0cafedf3>, line 20)

<details>
  <summary>Click to see answer</summary>
  <img alt="Smiley face" align="left" src="http://1.1m.yt/_FT_6m0.png">
  <p align='left'>Are you sure you tried to solve it on your own?</p>
      <pre>
          <code>
              1. alphabet = (list(string.ascii_lowercase) + list(string.digits) +
                                list(string.punctuation) + [' ', '\n'])
                 vocab_size = len(alphabet)

                 vocab = {}
                 for ix, t in enumerate(alphabet):
                     vocab[t] = ix
              2. for review in text:
                     temp.append([])
                     for i in review:
                         char = vocab.get(i,0)
                         if char != 0:
                             temp[-1].append(char)
              3. X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, value=-1)
                 X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH, value=-1)
          </code>
      </pre>

</details>

There are two ways of sentences char-representation. First we will overview the same approach as for words.

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D 
from keras.layers import Embedding 
from keras.layers import Dropout, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

EMBEDDING_DIM = 50

# initialize model
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH, trainable=True))
model.add(Conv1D(nb_filter=100, filter_length=4, border_mode='valid', activation='relu'))
model.add(Conv1D(nb_filter=100, filter_length=4, border_mode='valid', activation='relu'))
model.add(Conv1D(nb_filter=100, filter_length=4, border_mode='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Using TensorFlow backend.


NameError: name 'vocab_size' is not defined

**Embeddings**

The same as in previous example of word-based convolutional neural networks. We randomly initialise embedding weights and will improve them during model training.

**Convolutions**

We should experiment with it and try to find the best architecture for our task. 

**GlobalMaxPooling1D**

With GlobalMaxPooling1D we try to select the most important feature in each filter and pass it to the dense layer

**Dense**

It is fully-connected layers with dropout between them in case of regularization.

This architecture gives 78% on validation.

Another approach of char representation is One-Hot-Encoding. For example, our dictionary consists of 3 letters "a", "b", "c". OHE representation of "abca" will be $$a – 0 0 \\ b – 1 0 \\ c – 1 1 \\ a – 0 0$$

To do these transformations we will use Keras Model Api and some TensorFlow functions.

In [4]:
import tensorflow as tf
# ohe function
def ohe(x, sz):
    return tf.to_float(tf.one_hot(x, sz, on_value=1, off_value=0, axis=-1))

In [15]:
from keras.models import Model
from keras.layers import Input, Lambda
from keras.layers import MaxPooling1D
from keras.layers import LSTM


# define input
in_sentence = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
# Lambda Layer for ohe transformation
embedded = Lambda(ohe, output_shape=lambda x: (x[0], x[1], vocab_size), arguments={"sz": vocab_size})(in_sentence)
block = embedded
# convolutions with MaxPooling
for i in range(3):
    block = Conv1D(nb_filter=100, filter_length=4, border_mode='valid', activation='relu')(block)
    if i == 0:
        block = MaxPooling1D(pool_length=5)(block)
# LSTM cell
block = LSTM(128, dropout_U=0.1, dropout_W=0.1)(block)
block = Dense(100, activation='relu')(block)
block = Dense(2, activation='sigmoid')(block)
# gather model
model = Model(input=in_sentence, output=block)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 1024)          0                                            
____________________________________________________________________________________________________
lambda_3 (Lambda)                (None, 1024, 70)      0           input_4[0][0]                    
____________________________________________________________________________________________________
convolution1d_8 (Convolution1D)  (None, 1021, 100)     28100       lambda_3[0][0]                   
____________________________________________________________________________________________________
maxpooling1d_2 (MaxPooling1D)    (None, 204, 100)      0           convolution1d_8[0][0]            
___________________________________________________________________________________________

This model gives better result – 80% on validation set.

**Input**

Define input shape and its type.

**Lambda**

Custom Layer for OHE transfromations. Main arguments are function and output shape.

**Convolutions with MaxPooling**

Convolutions with MaxPooling select several best features and feed them to LSTM.

**LSTM**

<details>
  <summary>Why LSTM should be better than Dense Layer?</summary>
  <img alt="Smiley face" align="left" src="http://1.1m.yt/_FT_6m0.png">
  <p align='left'>Are you sure you tried to solve it on your own?</p>
      <pre>
            For Dense Layer we lose information about feautures order. LSTM can understand and learn it.
      </pre>

</details>

It was two main approaches. Let's try to build new architecture. ![image]()

In [31]:
from keras.layers import Merge, Flatten


in_sentence = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
embedded = Lambda(ohe, output_shape=lambda x: (x[0], x[1], vocab_size), arguments={"sz": vocab_size})(in_sentence)

conv_block = []
filters = [3,4,5]

# YOUR CODE

model = Model(input=in_sentence, output = result)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 1024)          0                                            
____________________________________________________________________________________________________
lambda_11 (Lambda)               (None, 1024, 70)      0           input_12[0][0]                   
____________________________________________________________________________________________________
convolution1d_61 (Convolution1D) (None, 1020, 100)     35100       lambda_11[0][0]                  
____________________________________________________________________________________________________
convolution1d_62 (Convolution1D) (None, 1020, 100)     35100       lambda_11[0][0]                  
___________________________________________________________________________________________

<details>
  <summary>Click to see answer</summary>
  <img alt="Smiley face" align="left" src="http://1.1m.yt/_FT_6m0.png">
  <p align='left'>Are you sure you tried to solve it on your own?</p>
      <pre>
            <code>
              for i in filters:
                  block = Conv1D(100, 5, activation='relu')(embedded)
                  conv_block.append(MaxPooling1D(pool_length=3)(block))
              merged_convs = Merge(mode='concat', concat_axis=1)(conv_block)
              conv_layer = MaxPooling1D(pool_length=5)(Conv1D(100, 5, activation='relu')(merged_convs))
              conv_layer = MaxPooling1D(pool_length=5)(Conv1D(100, 5, activation='relu')(conv_layer))
              flat_layer = Flatten()(conv_layer)
              result = Dense(2, activation='sigmoid')(flat_layer)
          </code>
      </pre>

</details>

Conclusions:
- char models could be very effective for text classification tasks
- better results when combinding cnn with lstm
- does not require a big vocabulary
- can be applied to any language
- avoid misspelling errors

ConvNets derive their name from the “convolution” operator. The primary purpose of Convolution in case of a ConvNet is to extract features from the input. Convolution preserves the spatial relationship between pixels by learning image features using small squares of input data. We will not go into the mathematical details of Convolution here, but will try to understand how it works over images.

Consider a 5 x 5 image whose pixel values are only 0 and 1 (note that for a grayscale image, pixel values range from 0 to 255, the green matrix below is a special case where pixel values are only 0 and 1):

![image](https://ujwlkarn.files.wordpress.com/2016/07/screen-shot-2016-07-24-at-11-25-13-pm.png?w=254&h=230)

Also, consider another 3 x 3 matrix as shown below:

![png](https://ujwlkarn.files.wordpress.com/2016/07/screen-shot-2016-07-24-at-11-25-24-pm.png?w=148&h=128)

Then, the Convolution of the 5 x 5 image and the 3 x 3 matrix can be computed as shown in the animation in figure below:

![animation](https://ujwlkarn.files.wordpress.com/2016/07/convolution_schematic.gif?w=536&h=392)

Take a moment to understand how the computation above is being done. We slide the orange matrix over our original image (green) by 1 pixel (also called ‘stride’) and for every position, we compute element wise multiplication (between the two matrices) and add the multiplication outputs to get the final integer which forms a single element of the output matrix (pink). Note that the 3×3 matrix “sees” only a part of the input image in each stride.

In CNN terminology, the 3×3 matrix is called a ‘filter‘ or ‘kernel’ or ‘feature detector’ and the matrix formed by sliding the filter over the image and computing the dot product is called the ‘Convolved Feature’ or ‘Activation Map’ or the ‘Feature Map‘. It is important to note that filters acts as feature detectors from the original input image.

In our case we have 